In [1]:
from google.colab import files
uploaded = files.upload()

Saving archive (5).zip to archive (5).zip


In [4]:
import zipfile
import os

# Replace 'your_file.zip' with the name of your uploaded file
zip_file_name = 'archive.zip'

# Extract
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('extracted')  # Extract to the 'extracted' folder

# List extracted files
os.listdir('extracted')


['The IQ-OTHNCCD lung cancer dataset', 'Test cases']

In [5]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch


In [6]:
data_dir = 'extracted/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset'
batch_size = 16
image_size = 224  # ResNet50 input size
num_classes = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Label mapping
class_mapping = {
    "Benign Cases": 0,
    "Normal Cases": 1,
    "Malignant Cases": 2
}


In [8]:
# Data transforms
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # Mean of ImageNet
                         [0.229, 0.224, 0.225])  # Std of ImageNet
])

In [9]:
# Dataset and loaders
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)

In [10]:
from torchvision import models
import torch.nn as nn

# Load pretrained ResNet
model = models.resnet50(pretrained=True)

# Modify final layer
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 197MB/s]


In [11]:
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [12]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [13]:
# Training loop
def train(model, loader):
    model.train()
    total_loss, correct = 0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()

    acc = 100 * correct / len(loader.dataset)
    print(f"Train Loss: {total_loss:.3f}, Accuracy: {acc:.2f}%")


In [14]:
# Validation loop
def evaluate(model, loader):
    model.eval()
    correct, preds_all, labels_all = 0, [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            preds_all.extend(preds.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())

    acc = 100 * np.sum(np.array(preds_all) == np.array(labels_all)) / len(labels_all)
    print(f"Validation Accuracy: {acc:.2f}%")
    print("Confusion Matrix:\n", confusion_matrix(labels_all, preds_all))
    print("Classification Report:\n", classification_report(labels_all, preds_all, target_names=class_mapping.keys()))


In [15]:
# Run
for epoch in range(1, 11):  # 10 epochs
    print(f"\nEpoch {epoch}")
    train(model, train_loader)
    evaluate(model, val_loader)


Epoch 1
Train Loss: 16.529, Accuracy: 88.83%
Validation Accuracy: 98.18%
Confusion Matrix:
 [[ 22   0   2]
 [  0 115   0]
 [  2   0  79]]
Classification Report:
                  precision    recall  f1-score   support

   Benign Cases       0.92      0.92      0.92        24
   Normal Cases       1.00      1.00      1.00       115
Malignant Cases       0.98      0.98      0.98        81

       accuracy                           0.98       220
      macro avg       0.96      0.96      0.96       220
   weighted avg       0.98      0.98      0.98       220


Epoch 2
Train Loss: 3.999, Accuracy: 97.95%
Validation Accuracy: 95.00%
Confusion Matrix:
 [[ 14   0  10]
 [  0 115   0]
 [  1   0  80]]
Classification Report:
                  precision    recall  f1-score   support

   Benign Cases       0.93      0.58      0.72        24
   Normal Cases       1.00      1.00      1.00       115
Malignant Cases       0.89      0.99      0.94        81

       accuracy                           0

In [16]:
torch.save(model.state_dict(), 'lung_cancer_model.pth')


In [19]:
# import os
# import torch
# from torchvision import transforms, models
# from PIL import Image

# # Setup
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# num_classes = 3
# model_path = 'lung_cancer_model.pth'
# test_dir = 'extracted/Test cases'  # Folder containing test images
# class_names = ['benign cases', 'malignant cases', 'normal cases']  # Adjust if order is different

# # Define transforms (same as training)
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406],
#                          [0.229, 0.224, 0.225])
# ])

# # Load model
# model = models.resnet18(pretrained=False)
# model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
# model.load_state_dict(torch.load(model_path, map_location=device))
# model = model.to(device)
# model.eval()

# # Predict on test images
# print("Predictions on Test Images:")
# for img_name in os.listdir(test_dir):
#     img_path = os.path.join(test_dir, img_name)
#     if img_path.lower().endswith(('.jpg', '.jpeg', '.png')):
#         image = Image.open(img_path).convert('RGB')
#         input_tensor = transform(image).unsqueeze(0).to(device)

#         with torch.no_grad():
#             output = model(input_tensor)
#             _, predicted = torch.max(output, 1)
#             label = class_names[predicted.item()]
#             print(f"{img_name}: {label}")
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model (use resnet50 if that was used during training)
model = models.resnet50(pretrained=False)
num_classes = 3  # change if your model has a different number of output classes
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("lung_cancer_model.pth", map_location=device))  # Your model path
model = model.to(device)
model.eval()

# Define transform (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Path to test images
test_folder = "extracted/Test cases"

# Class labels (modify according to your training dataset)
class_labels = ['benign cases', 'malignant cases', 'normal cases']

# Inference
for image_name in os.listdir(test_folder):
    image_path = os.path.join(test_folder, image_name)
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        predicted_class = class_labels[predicted.item()]

    print(f"{image_name}: {predicted_class}")


003412_03_01_108.png: malignant cases
000223_07_01_133.png: malignant cases
000247_05_01_221.png: malignant cases
000306_06_02_126.png: malignant cases
000226_02_01_079.png: malignant cases
000021_02_01_068.png: malignant cases
000228_03_03_165.png: malignant cases
000223_09_02_109.png: malignant cases
000198_05_01_124.png: malignant cases
000027_06_01_020.png: malignant cases
000103_13_01_019.png: malignant cases
000103_08_01_021.png: malignant cases
000265_11_02_190.png: malignant cases
000284_01_02_105.png: malignant cases
000306_06_02_201.png: malignant cases
000284_01_02_121.png: malignant cases
002463_01_01_029.png: malignant cases
000103_08_01_016.png: malignant cases
000247_04_01_165.png: malignant cases
000103_02_01_168.png: malignant cases
000103_05_01_136.png: malignant cases
004085_01_02_107.png: malignant cases
003311_02_01_028.png: malignant cases
000266_04_02_098.png: malignant cases
000103_04_02_090.png: malignant cases
000103_09_01_023.png: malignant cases
000058_07_01

In [26]:
import torch
from torchvision import models, transforms
from PIL import Image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Path to the trained model
model_path = 'lung_cancer_model.pth'  # Change this to your model's path

# Load the model
num_classes = 3
model = models.resnet50(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Standard ImageNet values
                         std=[0.229, 0.224, 0.225])
])

# Get image path from user
image_path = input("Enter image path: ")

# Load and preprocess the image
image = Image.open(image_path).convert("RGB")
input_tensor = transform(image).unsqueeze(0).to(device)

# Predict
with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)

print(f"Predicted class index: {predicted.item()}")


Enter image path: /content/extracted/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset/Malignant cases/Malignant case (179).jpg
Predicted class index: 1
